# Video Processing for BT Lab
Corrects png files
<br>
<br>
Notes:
1) looking for a specific path 
2) need to enter drive name
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
# import libraries
import os
import re

In [2]:
repo_dir = os.getcwd()
log_dir = repo_dir + "/records/logs"

In [3]:
def find_png_dir_from_log_file(log_file:str):
    png_directories = []
    
    with open(log_file, "r") as log_data:
        log_files = log_data.read().split("\n")
    
    for log_file in log_files:
        log_file_temp = log_file.replace("\\", "/")
        log_file_split = log_file_temp.split("/")
        
        if "" in log_file_split:
            log_file_split.remove("")

        video_folder = "/".join(log_file_split[:-1])

        if video_folder not in png_directories:
            png_directories.append(video_folder)

    return png_directories

In [ ]:
def save_folder(list_of_paths:list, log_filename) -> None:
    if os.path.exists(log_filename):
        os.remove(log_filename)
    
    with open(log_filename, "a") as log_data:
        for folder in list_of_paths:
            log_data.write(f"{folder}\n")

## Retrieve all videos (stage 3.2)

Correct png files

In [4]:
"get png folders (rgb)"
log_filename = log_dir +"/DI_CAMERA_P3225-png.log"
logfile_save = log_dir + "/rgb_folders-png.log"
rgb_png_folders = find_png_dir_from_log_file(log_filename)


C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/frames_Arun2_lower_bound



In [5]:
"get png folders (thermal)"
log_filename = log_dir +"/DI_THERMAL_FLIR-png.log"
thermal_png_folders = find_png_dir_from_log_file(log_filename)

for folder in thermal_png_folders:
    print(folder)

C:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_1/2 meters/H/15_holding_blanket_2meters/frames
C:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_2/15_2nd_2meters/15_2nd_hold_blanket_2meters/frames/frames
C:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_2/15_2nd_2meters/15_2nd_relaxed_blanket_2meters/frames/frames
C:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_2/15_2nd_3meters/blanket/holding/frames
C:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_2/15_2nd_3meters/blanket/relaxed/frames
C:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_2/15_2nd_3meters/without/holding/frames
C:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_2/15_2nd_3meters/without/relaxed/frames

